# MULTI-LAYER PERCEPTRON ON MNIST

In [1]:
import numpy as np
# import tensorflow as tf
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
%matplotlib inline  
print ("PACKAGES LOADED")

PACKAGES LOADED


# LOAD MNIST

In [2]:
def OnehotEncoding(target):
    from sklearn.preprocessing import OneHotEncoder
    target_re = target.reshape(-1,1)
    enc = OneHotEncoder()
    enc.fit(target_re)
    return enc.transform(target_re).toarray()

def SuffleWithNumpy(data_x, data_y):
    idx = np.random.permutation(len(data_x))
    x,y = data_x[idx], data_y[idx]
    return x,y

In [3]:
print ("Download and Extract MNIST dataset")
# mnist = input_data.read_data_sets('data/', one_hot=True)
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

print
print (" tpye of 'mnist' is %s" % (type(mnist)))
print (" number of train data is %d" % (len(x_train)))
print (" number of test data is %d" % (len(x_test)))
num_train_data = len(x_train)
trainimg   = x_train
trainimg = trainimg.reshape(len(trainimg),784)
trainlabel = OnehotEncoding(y_train)
testimg    = x_test
testimg = testimg.reshape(len(testimg),784)
testlabel  = OnehotEncoding(y_test)
print ("MNIST loaded")
tf.disable_eager_execution()

Download and Extract MNIST dataset
 tpye of 'mnist' is <class 'tensorflow.python.util.module_wrapper.TFModuleWrapper'>
 number of train data is 60000
 number of test data is 10000
MNIST loaded


# DEFINE MODEL

In [4]:
# NETWORK TOPOLOGIES
n_hidden_1 = 256 
n_hidden_2 = 128 
n_input    = 784 
n_classes  = 10  

# INPUTS AND OUTPUTS
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])
    
# NETWORK PARAMETERS
stddev = 0.1
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=stddev)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=stddev)),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes], stddev=stddev))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}
print ("NETWORK READY")

NETWORK READY


# MLP AS A FUNCTION

In [5]:
def multilayer_perceptron(_X, _weights, _biases):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1'])) 
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, _weights['h2']), _biases['b2']))
    return (tf.matmul(layer_2, _weights['out']) + _biases['out'])

# DEFINE FUNCTIONS

In [7]:
# PREDICTION
pred = multilayer_perceptron(x, weights, biases)

# LOSS AND OPTIMIZER
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels= y)) 
# optm = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(cost) 
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost) 
corr = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))    
accr = tf.reduce_mean(tf.cast(corr, "float"))

# INITIALIZER
init = tf.global_variables_initializer()
print ("FUNCTIONS READY")

FUNCTIONS READY


# RUN

In [8]:
# PARAMETERS
training_epochs = 20
batch_size      = 100
display_step    = 4
# LAUNCH THE GRAPH
sess = tf.Session()
sess.run(init)
# OPTIMIZE
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(num_train_data/batch_size)
    # ITERATION
    for i in range(total_batch):
        batch_xs=trainimg[i*batch_size:(i+1)*batch_size]
        batch_ys=trainlabel[i*batch_size:(i+1)*batch_size]
        
        feeds = {x: batch_xs, y: batch_ys}
        sess.run(optm, feed_dict=feeds)
        avg_cost += sess.run(cost, feed_dict=feeds)
    avg_cost = avg_cost / total_batch
    # DISPLAY
    if (epoch+1) % display_step == 0:
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
        feeds = {x: batch_xs, y: batch_ys}
        train_acc = sess.run(accr, feed_dict=feeds)
        print ("TRAIN ACCURACY: %.3f" % (train_acc))
        feeds = {x: testimg, y: testlabel}
        test_acc = sess.run(accr, feed_dict=feeds)
        print ("TEST ACCURACY: %.3f" % (test_acc))
    trainimg,trainlabel = SuffleWithNumpy(trainimg,trainlabel)
print ("OPTIMIZATION FINISHED")

Epoch: 003/020 cost: 0.116514339
TRAIN ACCURACY: 0.990
TEST ACCURACY: 0.966
Epoch: 007/020 cost: 0.044108750
TRAIN ACCURACY: 0.990
TEST ACCURACY: 0.976
Epoch: 011/020 cost: 0.017314874
TRAIN ACCURACY: 1.000
TEST ACCURACY: 0.979
Epoch: 015/020 cost: 0.006136829
TRAIN ACCURACY: 1.000
TEST ACCURACY: 0.981
Epoch: 019/020 cost: 0.002604427
TRAIN ACCURACY: 1.000
TEST ACCURACY: 0.981
OPTIMIZATION FINISHED
